In [1]:
import sqlalchemy as sa
import os
import sys
import urllib.parse as url
import pandas as pd

In [2]:
username = os.getlogin()
external_folder_path = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Custom_Python_Functions/'
sys.path.append(external_folder_path)
from custom_python_functions import create_connection, load_key, decrypt

key1 = 'user_key.ky'
key_file1 = 'user_key.txt'
key2 = 'pass_key.ky'
key_file2 = 'pass_key.txt'

key1 = load_key(external_folder_path, key1)
uid = decrypt(external_folder_path, key_file1, key1)

key2 = load_key(external_folder_path, key2)
passwd = decrypt(external_folder_path, key_file2, key2)

# Setup connection parameters
server = 'danvuk.database.windows.net'
dbase = 'Financial_Securities'

# Create a connection to the database
s, e = create_connection(server, dbase, uid, passwd)
s1 = s()  # Instantiate a session object


In [3]:
Base = sa.orm.declarative_base()

class Yahoo_Equity_Prices(Base):
    
    """
    SQLAlchemy ORM class representing the 'Yahoo_Equity_Prices' table in the 'Equities' schema.

    Attributes:
    __tablename__ (str): The name of the table in the database.
    __table_args__ (dict): Additional arguments for the table, including schema name.
    Date (Column): The date of the equity price record; part of the composite primary key.
    Ticker_ID (Column): A unique identifier for each equity; part of the composite primary key.
    Open (Column): The opening price of the equity on the given date.
    High (Column): The highest price of the equity on the given date.
    Low (Column): The lowest price of the equity on the given date.
    Close (Column): The closing price of the equity on the given date.
    Volume (Column): The trading volume of the equity on the given date.
    """
    __tablename__ = 'Yahoo_Equity_Prices'
    __table_args__ = {'schema': 'Equities'}
    Date=sa.Column('Date', sa.Date, primary_key=True)
    Ticker_ID=sa.Column('Ticker_ID', sa.Integer, primary_key=True)
    Open=sa.Column('Open', sa.Float)
    High=sa.Column('High', sa.Float)
    Low=sa.Column('Low', sa.Float)
    Close=sa.Column('Close', sa.Float)
    Volume=sa.Column('Volume', sa.BigInteger)

In [4]:
# SQL query to get the Ticker_ID, Ticker and pricing data from Data_STG and Equities tables
# and joined with Market_Calendar to get all possible dates in case there are missing dates
# between the bounds of the existing pricing dates for each Ticker.
sql_stat = """WITH q1 AS 
(SELECT
 Description, 
 CAST(MIN(Date) AS Date) AS Min_Date,
 CAST(MAX(Date) AS Date) AS Max_Date
FROM [Financial_Securities].[Equities].[Data_STG]
GROUP BY Description),
q2 AS
(SELECT
 q3.Ticker_ID,
 q3.Ticker,
 q4.Date
FROM
[Financial_Securities].[Equities].[Equities] q3,
[Financial_Securities].[Equities].[Market_Calendar] q4),
q5 AS
(SELECT 
 q2.Ticker_ID,
 q2.Ticker,
 q2.Date 
FROM q2
INNER JOIN q1
ON q1.Description = q2.Ticker
AND q2.Date BETWEEN q1.Min_Date AND q1.Max_Date)
SELECT 
 q5.Ticker_ID,
 q5.Ticker,
 q5.Date,
 ROUND(q7.Float_Value1, 2) AS Float_Value1,
 ROUND(q7.Float_Value2, 2) AS Float_Value2, 
 ROUND(q7.Float_Value3, 2) AS Float_Value3, 
 ROUND(q7.Float_Value4, 2) AS Float_Value4, 
 q7.Int_Value1
FROM q5
LEFT OUTER JOIN [Financial_Securities].[Equities].[Data_STG] q7
ON q5.Ticker = q7.Description
AND q5.Date = CAST(q7.Date AS Date)
ORDER BY q5.Ticker_ID, q5.Date
"""
                                                                    
try:              
    df_pricing = pd.read_sql(sql_stat, s1.bind) # Execute the SQL query through the session and bind the data to the df_pricing dataframe
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:
    print(f"Issue querying database tables! Error: {e}")
    s1.close()  # Close the session
    raise  # Re-raise the exception to propagate the error
    
print("Query data load is complete")

Query data load is complete


In [5]:
# Let's sort and forward fill any pricing data that is missing for dates in the
# Market Calendar within the bounds of the existing pricing dates for each Ticker
df_pricing.sort_values(by=['Ticker_ID', 'Date'], inplace=True)
df_pricing[['Float_Value1', 'Float_Value2', 'Float_Value3', 'Float_Value4']] = df_pricing.groupby('Ticker_ID')[['Float_Value1', 'Float_Value2', 'Float_Value3', 'Float_Value4']].ffill()

In [6]:
for index, row in df_pricing.iterrows():
    try:
        
        # Query the 'Yahoo_Equity_Prices' table to find records where the 'Date' column matches the value in the DataFrame's 'Date' row
        # and the 'Ticker_ID' column matches the value in the DataFrame's 'Ticker_ID' row
        q1 = s1.query(Yahoo_Equity_Prices).filter(Yahoo_Equity_Prices.Date == row.Date, Yahoo_Equity_Prices.Ticker_ID == row.Ticker_ID)

        # Check if any records were found with the specified 'Date' and 'Ticker_ID'
        if (q1.count() >= 1):
            # If one or more records are found, get the first matching record
            q1 = s1.query(Yahoo_Equity_Prices).filter(Yahoo_Equity_Prices.Date == row.Date, Yahoo_Equity_Prices.Ticker_ID == row.Ticker_ID).first()
            # Update the pricing attributes of the found record with the values from the DataFrame's pricing columns
            q1.Open=row['Float_Value1']
            q1.High=row['Float_Value2']
            q1.Low=row['Float_Value3']
            q1.Close= row['Float_Value4']
            q1.Volume=row['Int_Value1']
            
        else:
            
            # Create a new Yahoo_Equity_Prices object for each row in df_pricing dataframe
            q1 = Yahoo_Equity_Prices(
                Ticker_ID=row['Ticker_ID'],
                Date=row['Date'],
                Open=row['Float_Value1'],
                High=row['Float_Value2'],
                Low=row['Float_Value3'],
                Close=row['Float_Value4'],
                Volume=row['Int_Value1']
            )
    
            s1.add(q1)  # Add the instance to the session
        
    # Handle SQLAlchemy errors if they occur during adding the object
    except sa.exc.SQLAlchemyError as e:
        message = f"Issue with updating Yahoo_Equity_Prices database table for Ticker: {row.Ticker}. Error: {e}"
        print(message)
        s1.close()  # Close the session
        raise  # Re-raise the exception to propagate the error

s1.commit() # Commit the transactions to the database

print("Database data load is complete")


Database data load is complete


In [7]:
# SQL query to count the number of records in the Data_STG table
sql_stat2 = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Data_STG]"""
          
try: 
    result1 = e.execute(sql_stat2)  # Execute the count query
    cnt_recs1 = result1.scalar()  # Get the count of records
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:
    print(f"Issue querying Data_STG database table for count! Error: {e}")


# SQL query to count the number of records in the Yahoo_Equity_Prices table
sql_stat3 = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Yahoo_Equity_Prices]"""
              
try: 
    result1 = e.execute(sql_stat3)  # Execute the count query
    cnt_recs2 = result1.scalar()  # Get the count of records
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:   
    print(f"Issue querying Sub_Industries database table for count! Error: {e}")

# Compare the record counts and print the result
if cnt_recs2 < cnt_recs1:
    print(f"Only {cnt_recs2} of {cnt_recs1} records were loaded into Yahoo_Equity_Prices database table!")
else:
    print(f"All {cnt_recs2} records were loaded into Yahoo_Equity_Prices database table!") 
    

All 465762 records were loaded into Yahoo_Equity_Prices database table!


In [8]:
s1.close()  # Close the session